In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

# fractal

> Fill in a module description here

In [ ]:
#| default_exp fractal

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from abc import abstractmethod
import numpy as np
from fractalart.core import Image

In [ ]:
#| export
class Fractal(Image):
    def __init__(self, x_min: float = -2.0, x_max: float = 1.0, y_min: float = -1.5, y_max: float = 1.5, max_iter: int = 1000):
        self._x_min = x_min
        self._x_max = x_max
        self._y_min = y_min
        self._y_max = y_max
        if max_iter <= 0:
            raise ValueError("max_iter and height must be positive integers.")
        self._max_iter = max_iter

    @abstractmethod
    def compute(self) -> np.ndarray:
        pass

    @property
    def max_iter(self) -> int:
        return self._max_iter

    @max_iter.setter
    def max_iter(self, value: int):
        if value <= 0:
            raise ValueError("max_iter must be a positive integer.")
        self._max_iter = value

    def plot(self, cmap: str = "turbo"):
        """Plot the rendered image using matplotlib."""
        plt.figure(figsize=(10, 10))
        plt.imshow(self._image, cmap=cmap, extent=[self._x_min, self._x_max, self._y_min, self._y_max])
        plt.show()

    def set_zoom(self, zoom: float, center: tuple[float, float]):
        delta = 2 / zoom
        self._x_min, self._y_min = center[0] - delta, center[1] - delta
        self._x_max, self._y_max = center[0] + delta, center[1] + delta

    def render(self) -> np.ndarray:
        width, height = self.resolution
        return self.compute()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()